In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Conv1D, MaxPooling1D, Attention
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report

import torch
from torch import nn, optim

from collections import defaultdict

In [2]:
# full_df = pd.read_csv('data/affcon_final.csv')

In [3]:
# # Full data 
# full_df = pd.read_csv('data/full_diplomacy_data.csv')
# full_df = full_df.rename(columns={"text": "Input.full_text", "meta.deception_quadrant": "Input.deception_quadrant"})

In [4]:
# full_df = full_df[full_df['meta.speaker_intention'] == 'Truth']

In [5]:
# full_df.head()

In [7]:
full_df = pd.read_csv('data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
full_df = full_df.dropna() # dataset contains NaN values, dropping NaNs here

X = full_df['Input.full_text']

# full_df["Input.deception_quadrant"] = full_df["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y = full_df['Input.deception_quadrant']
#y = full_df['Answer.3rapport.yes_label']
#y = full_df['Answer.4shareinformation.yes_label']
# y = full_df['Answer.2reasoning.yes_label']
#y = full_df['Answer.1gamemove.yes_label']

le = LabelEncoder() # this can convert our categories into labels, make sure you don't have NaNs or Nulls in your data first
y = le.fit_transform(y)
print(y.shape)

# we reshape 
y = y.reshape(-1,1) # the -1 allows it to have whatever number went in there
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11366,)
(11366, 1)
(9092,)
(9092, 1)
(2274,)
(2274, 1)


In [8]:
max_words = 1000
max_len = 100

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

sequences = tok.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences,maxlen=max_len)
# X_train = sequence.pad_sequences(sequences)

In [9]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## LSTM Model

In [7]:
Inp = Input(name='inputs',shape=[max_len])
x = Embedding(max_words,50,input_length=max_len)(Inp)
x = LSTM(64,name='LSTM_01')(x)
x = Dropout(0.5,name='Dropout')(x)
x = Dense(128,activation='relu',name='Dense_01')(x)
# x = Dropout(0.5,name='Dropout')(x)
out = Dense(1,activation='sigmoid', name='output')(x)

model = Model(inputs=Inp,outputs=out)

model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           50000     
_________________________________________________________________
LSTM_01 (LSTM)               (None, 64)                29440     
_________________________________________________________________
Dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
Dense_01 (Dense)             (None, 128)               8320      
_________________________________________________________________
output (Dense)               (None, 1)                 129       
Total params: 87,889
Trainable params: 87,889
Non-trainable params: 0
__________________________________________________

In [8]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

model.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

Epoch 1/15
74/74 [==============================] - 2s 26ms/step - loss: 0.2102 - acc: 0.9474 - f1_m: 0.9705 - precision_m: 0.9566 - recall_m: 0.9905 - val_loss: 0.1742 - val_acc: 0.9585 - val_f1_m: 0.9791 - val_precision_m: 0.9592 - val_recall_m: 1.0000
Epoch 2/15
74/74 [==============================] - 1s 19ms/step - loss: 0.1824 - acc: 0.9567 - f1_m: 0.9778 - precision_m: 0.9568 - recall_m: 1.0000 - val_loss: 0.1712 - val_acc: 0.9585 - val_f1_m: 0.9791 - val_precision_m: 0.9592 - val_recall_m: 1.0000
Epoch 3/15
74/74 [==============================] - 1s 19ms/step - loss: 0.1743 - acc: 0.9567 - f1_m: 0.9778 - precision_m: 0.9567 - recall_m: 1.0000 - val_loss: 0.1766 - val_acc: 0.9585 - val_f1_m: 0.9791 - val_precision_m: 0.9592 - val_recall_m: 1.0000


In [9]:
test_sequences_LSTM = tok.texts_to_sequences(X_test)
X_test_LSTM = sequence.pad_sequences(test_sequences_LSTM,maxlen=max_len)
model.evaluate(X_test_LSTM,y_test)

93/93 [==============================] - 1s 7ms/step - loss: 0.2039 - acc: 0.9486 - f1_m: 0.9734 - precision_m: 0.9489 - recall_m: 1.0000


[0.20389054715633392,
 0.9485617876052856,
 0.9733641147613525,
 0.948924720287323,
 1.0]

In [10]:
a = model.predict(X_test_LSTM).round()
precision_recall_fscore_support(y_test, a, average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4742808798646362, 0.5, 0.4868009725599166, None)

## LSTM Model with Attention

In [9]:
SINGLE_ATTENTION_VECTOR = False
APPLY_ATTENTION_BEFORE_LSTM = False
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    # output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [10]:
def model_attention_applied_after_lstm():
    #inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    lstm_units = 64
    lstm_out = LSTM(lstm_units, return_sequences=True)(layer)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model([inputs], output)
    return model

In [11]:
from keras.layers import merge
from keras.layers import multiply
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

from keras.utils.vis_utils import plot_model

In [12]:
INPUT_DIM = 50
TIME_STEPS = max_len
m = model_attention_applied_after_lstm()
m.summary()
m.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 50)      50000       inputs[0][0]                     
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 100, 64)      29440       embedding[0][0]                  
__________________________________________________________________________________________________
permute (Permute)               (None, 64, 100)      0           lstm[0][0]                       
_______________________________________________________________________________________

In [13]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

m.fit(X_train,y_train,
      batch_size=128,
      epochs=15,
      validation_split=0.2,
      callbacks=[early_stop])

Epoch 1/15
57/57 [==============================] - 1s 22ms/step - loss: 0.3215 - acc: 0.9293 - f1_m: 0.9633 - precision_m: 0.9293 - recall_m: 1.0000 - val_loss: 0.2269 - val_acc: 0.9401 - val_f1_m: 0.9675 - val_precision_m: 0.9374 - val_recall_m: 1.0000
Epoch 2/15
57/57 [==============================] - 1s 14ms/step - loss: 0.2561 - acc: 0.9293 - f1_m: 0.9632 - precision_m: 0.9293 - recall_m: 1.0000 - val_loss: 0.2278 - val_acc: 0.9401 - val_f1_m: 0.9675 - val_precision_m: 0.9374 - val_recall_m: 1.0000


In [14]:
test_sequences_LSTM = tok.texts_to_sequences(X_test)
X_test_LSTM = sequence.pad_sequences(test_sequences_LSTM,maxlen=max_len)
m.evaluate(X_test_LSTM,y_test)

72/72 [==============================] - 0s 3ms/step - loss: 0.2591 - acc: 0.9279 - f1_m: 0.9624 - precision_m: 0.9288 - recall_m: 1.0000


[0.2591043710708618,
 0.9278804063796997,
 0.9624230861663818,
 0.9288194179534912,
 1.0]

In [15]:
a = m.predict(X_test_LSTM).round()
precision_recall_fscore_support(y_test, a, average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.46394019349164467, 0.5, 0.48129562043795615, None)

## CNN Model

In [38]:
model_CNN = Sequential(name="CNN_with_embeddings")
model_CNN.add(Embedding(max_words, 50, input_length=max_len))
model_CNN.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model_CNN.add(MaxPooling1D(pool_size=2))
model_CNN.add(Flatten())
model_CNN.add(Dropout(0.5))
model_CNN.add(Dense(10, activation='relu'))
model_CNN.add(Dense(1, activation='sigmoid'))

model_CNN.compile(loss='binary_crossentropy', 
              optimizer= 'adam',
              metrics=['acc',f1_m,precision_m, recall_m])

model_CNN.summary()

Model: "CNN_with_embeddings"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 93, 32)            12832     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 46, 32)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1472)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1472)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                14730     
_________________________________________________________________
dense_13 (Dense)             (None, 1)         

In [39]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

model_CNN.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

Epoch 1/15
57/57 [==============================] - 1s 15ms/step - loss: 0.2823 - acc: 0.9372 - f1_m: 0.9621 - precision_m: 0.9534 - recall_m: 0.9830 - val_loss: 0.1963 - val_acc: 0.9505 - val_f1_m: 0.9738 - val_precision_m: 0.9492 - val_recall_m: 1.0000
Epoch 2/15
57/57 [==============================] - 1s 11ms/step - loss: 0.1926 - acc: 0.9524 - f1_m: 0.9755 - precision_m: 0.9523 - recall_m: 1.0000 - val_loss: 0.2001 - val_acc: 0.9505 - val_f1_m: 0.9738 - val_precision_m: 0.9492 - val_recall_m: 1.0000


In [34]:
test_sequences_CNN = tok.texts_to_sequences(X_test)
X_test_CNN = sequence.pad_sequences(test_sequences_CNN,maxlen=max_len)

model_CNN.evaluate(X_test_CNN,y_test)

72/72 [==============================] - 0s 3ms/step - loss: 0.4295 - acc: 0.8465 - f1_m: 0.9166 - precision_m: 0.8485 - recall_m: 1.0000


[0.42947834730148315,
 0.8465259671211243,
 0.9165847301483154,
 0.8485243320465088,
 1.0]

In [40]:
a_cnn = model_CNN.predict(X_test_CNN).round()
precision_recall_fscore_support(y_test, a_cnn, average='macro')

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4731750219876869, 0.5, 0.48621780388612745, None)